In [1]:
import numpy as np
from numpy import *

In [463]:
# Explanation of data
#
#    The death rate is to be represented as a function of other variables.
#
#    There are 60 rows of data. Each point has 15 features:
#
#      A1,  the average annual precipitation;
#      A2,  the average January temperature;
#      A3,  the average July temperature;
#      A4,  the size of the population older than 65;
#      A5,  the number of members per household;
#      A6,  the number of years of schooling for persons over 22;
#      A7,  the number of households with fully equipped kitchens;
#      A8,  the population per square mile; 
#      A9,  the size of the nonwhite population;
#      A10, the number of office workers;
#      A11, the number of families with an income less than $3000;
#      A12, the hydrocarbon pollution index;
#      A13, the nitric oxide pollution index;
#      A14, the sulfur dioxide pollution index;
#      A15, the degree of atmospheric moisture.
#
#    The output is:
#      Y,   the death rate.
#      A and y; last column is y

In [464]:
import pandas as pd
# Upload data.
df = pd.DataFrame(pd.read_csv('deathrate_instance_python.dat', header=None, sep="\s+"))
df.columns = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'Y']

In [465]:
# Define matrix X and vector Y.
Y = df['Y']
X = df.drop(columns=['Y'])

In [466]:
# Add a column of ones.
X = pd.concat([X, pd.DataFrame({'O': np.ones(len(X))})], axis=1) 

In [467]:
# Transform dataframe into matrix.
Ymat = Y.values
Xmat = X.values

In [468]:
# Optimal values by AMPL.
wopt = np.array([0.48824,-0.0463881,0.102978,-0.0377061,0.00490736,-0.0339172,-0.255786,0.00564845,0.649708,-0.12622,0.213407,-0.207831,0.109842,0.376641,0.00995978])
gammaopt = 895.141
# Define it as 1 vector (list):
wopt = [0.48824,-0.0463881,0.102978,-0.0377061,0.00490736,-0.0339172,-0.255786,0.00564845,0.649708,-0.12622,0.213407,-0.207831,0.109842,0.376641,0.00995978,895.141]

In [469]:
"""
MINOS 5.51: optimal solution found.
133 iterations, objective 61484.65464
Nonlin evals: obj = 361, grad = 360, constrs = 361, Jac = 360.

norm2_w = -10785.1
gamma = 895.141

w [*] :=
 1   0.48824
 2  -0.0463881
 3   0.102978
 4  -0.0377061
 5   0.00490736
 6  -0.0339172
 7  -0.255786
 8   0.00564845
 9   0.649708
10  -0.12622
11   0.213407
12  -0.207831
13   0.109842
14   0.376641
15   0.00995978

"""
print()

In [470]:
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

In [471]:
# Loss func loss(x)
def loss_func (x):
    """
    Objective function: loss / cost function.
    
    Parameters
    ----------
    x array of weights and gamma.
    
    Returns
    -------
    The objective function loss(x) evaluated at x.
    
    """
    w = np.asarray(x).T
    
    cost = (1/2) * ((Xmat @ w) - Ymat).T @ ((Xmat @ w) - Ymat)
  
    return np.squeeze(cost)

In [472]:
loss_func(wopt)

61484.65071643571

In [473]:
def loss_grad (x): 
    """
    Gradient of the objective function (loss / cost function).
    
    Parameters
    ----------
    x array of weights and gamma.
    
    Returns
    -------
    Array of the gradient of the objective function loss(x) 
    evaluated at x.
    
    """
    w = np.asarray(x).T
    
    return Xmat.T @ (Xmat @ w - Ymat)

In [474]:
loss_grad(wopt) # valor optim

array([-1.05322859e+04,  9.99806306e+02, -2.22295893e+03,  8.13127974e+02,
       -1.05927484e+02,  7.31350810e+02,  5.51550115e+03, -2.10784096e+02,
       -1.40146068e+04,  2.72154380e+03, -4.60357165e+03,  4.48211610e+03,
       -2.36983501e+03, -8.12547179e+03, -2.16150552e+02, -2.28754456e-02])

In [475]:
def loss_hess (x):
    """
    Hessian matrix of the objective function (loss / cost function).
    
    Parameters
    ----------
    x array of weights and gamma.
    
    Returns
    -------
    Matrix of the Hessian of the objective function loss(x) 
    evaluated at x.
    
    """
    w = np.asarray(x).T
    
    return Xmat.T @ Xmat

In [476]:
#loss_hess(wopt)

In [570]:
def cons_func (x):
    """
    Constraint.
    
    Parameters
    ----------
    x array of weights and gamma.
    Gamma will be ignored (x[-1]).
    
    Returns
    -------
    The constraint evaluated at x.
    
    """
    
    w = np.asarray(x).T
    w = w[:-1]
    return np.sum(w**2)

In [571]:
cons_func(wopt)

1.0000003642408806

In [588]:
def cons_jacobian (x):
    """
    Gradient of the constraint.
    
    Parameters
    ----------
    x array of weights and gamma.
    Gamma will be ignored (x[-1]).
    
    Returns
    -------
    The gradient of the constraint evaluated at x.
    
    """
    
    w = np.asarray(x).T
    w = w[:-1]
    w = np.append(w, 0)
    u = (2*w).tolist()
    return u

In [590]:
cons_jacobian(wopt)

[0.97648,
 -0.0927762,
 0.205956,
 -0.0754122,
 0.00981472,
 -0.0678344,
 -0.511572,
 0.0112969,
 1.299416,
 -0.25244,
 0.426814,
 -0.415662,
 0.219684,
 0.753282,
 0.01991956,
 0.0]

In [603]:
def cons_hess (x, w2):
    """
    Product of the hessian of the constraint by an arbitrary vector.
    It is needed for the minimize function.
    
    Parameters
    ----------
    x array of weights and gamma.
    Gamma will be ignored (x[-1]).
    v vector of length = number of constraints.
    
    Returns
    -------
    The Hessian of the constraint evaluated at x by v[0].
    
    """
    
    #x[-1] = 0
    w = np.asarray(x).T
    #H = (np.eye(w.shape[0])*2)
    #H[-1,-1] = 0
    H = np.zeros((w.shape[0],w.shape[0]))
    np.fill_diagonal(H,2)
    H[-1,-1] = 0
    return w2 * H
    #return v * H

In [604]:
# cons_hess (w, np.ones(16))

In [605]:
# Define hyperparamater t:
t = 1

## OPTIMIZATION ##
# Assign initial random weights:
w = np.squeeze(np.random.rand(1,16))
w = np.ones(16)

# Define non linear constraint:
nonlinear_constraint = NonlinearConstraint(cons_func, -np.inf, t, jac = cons_jacobian, hess = cons_hess) 
# nonlinear_constraint = NonlinearConstraint(cons_func, -np.inf, t) 

# Solve:
sol = minimize(loss_func, w, method='trust-constr', jac=loss_grad, hess=loss_hess,
               constraints=[nonlinear_constraint], options={'verbose': 1})
# Output:
print(sol)

`xtol` termination condition is satisfied.
Number of iterations: 144, function evaluations: 191, CG iterations: 794, optimality: 3.97e-05, constraint violation: 0.00e+00, execution time: 0.27 s.
 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 794
      cg_stop_cond: 4
            constr: [array([1.])]
       constr_nfev: [191]
       constr_nhev: [91]
       constr_njev: [86]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.2699158191680908
               fun: 61484.65464046687
              grad: array([-1.05314313e+04,  1.00060115e+03, -2.22125180e+03,  8.13329504e+02,
       -1.05852785e+02,  7.31601887e+02,  5.51735331e+03, -1.21838297e+02,
       -1.40143362e+04,  2.72259783e+03, -4.60324321e+03,  4.48296568e+03,
       -2.36932573e+03, -8.12422725e+03, -2.14833708e+02,  1.38982657e-05])
               jac: [array([[ 0.9764791 , -0.09277619,  0.20595548, -0.07541228,  0.00981472,
        -0.06783446, -0.5

In [599]:
sol = minimize(loss_func, wopt, method='trust-constr', jac=loss_grad, hess=loss_hess,options={'verbose': 1})

`xtol` termination condition is satisfied.
Number of iterations: 662, function evaluations: 651, CG iterations: 9947, optimality: 2.11e-01, constraint violation: 0.00e+00, execution time:  3.4 s.


In [600]:
print(sol)

         cg_niter: 9947
     cg_stop_cond: 4
           constr: []
      constr_nfev: []
      constr_nhev: []
      constr_njev: []
   constr_penalty: 1.0
 constr_violation: 0
   execution_time: 3.4214839935302734
              fun: 23000.394572189656
             grad: array([ 0.01861397, -0.01730387,  0.02112825, -0.00488185,  0.0019099 ,
        0.00047303, -0.01116767,  0.21065525,  0.01809868,  0.00265811,
       -0.01319207,  0.00106848,  0.00710287,  0.00321044, -0.00560311,
       -0.0122878 ])
              jac: []
  lagrangian_grad: array([ 0.01861397, -0.01730387,  0.02112825, -0.00488185,  0.0019099 ,
        0.00047303, -0.01116767,  0.21065525,  0.01809868,  0.00265811,
       -0.01319207,  0.00106848,  0.00710287,  0.00321044, -0.00560311,
       -0.0122878 ])
          message: '`xtol` termination condition is satisfied.'
           method: 'equality_constrained_sqp'
             nfev: 651
             nhev: 650
              nit: 662
            niter: 662
           

In [611]:
yo = np.round(array([ 4.88239551e-01, -4.63880973e-02,  1.02977740e-01, -3.77061410e-02,
        4.90736021e-03, -3.39172299e-02, -2.55785757e-01,  5.64845110e-03,
        6.49707815e-01, -1.26220254e-01,  2.13407404e-01, -2.07831310e-01,
        1.09842480e-01,  3.76641024e-01,  9.95974041e-03,  8.95141404e+02]),4)
print(yo)

[ 4.882000e-01 -4.640000e-02  1.030000e-01 -3.770000e-02  4.900000e-03
 -3.390000e-02 -2.558000e-01  5.600000e-03  6.497000e-01 -1.262000e-01
  2.134000e-01 -2.078000e-01  1.098000e-01  3.766000e-01  1.000000e-02
  8.951414e+02]


In [629]:
8.95141404e+02

895.141404

In [630]:
np.linalg.norm(array([ 0.01861397, -0.01730387,  0.02112825, -0.00488185,  0.0019099 ,
        0.00047303, -0.01116767,  0.21065525,  0.01809868,  0.00265811,
       -0.01319207,  0.00106848,  0.00710287,  0.00321044, -0.00560311,
       -0.0122878 ]))

0.21534461815763473

In [636]:
# Define hyperparamater t:
t = 1

## OPTIMIZATION ##
# Assign initial random weights:
w = np.squeeze(np.random.rand(1,16))
w = np.ones(16)

# Define non linear constraint:
nonlinear_constraint = NonlinearConstraint(cons_func, -np.inf, t, jac = cons_jacobian, hess = cons_hess) 
# nonlinear_constraint = NonlinearConstraint(cons_func, -np.inf, t) 

# Solve:
sol = minimize(loss_func, w, method='trust-constr', jac=loss_grad, hess=loss_hess, options={'verbose': 1})
# Output:
print(sol)

`xtol` termination condition is satisfied.
Number of iterations: 437, function evaluations: 426, CG iterations: 6106, optimality: 8.34e-02, constraint violation: 0.00e+00, execution time:  2.3 s.
         cg_niter: 6106
     cg_stop_cond: 4
           constr: []
      constr_nfev: []
      constr_nhev: []
      constr_njev: []
   constr_penalty: 1.0
 constr_violation: 0
   execution_time: 2.34763503074646
              fun: 23000.38378504645
             grad: array([ 0.00508902,  0.000236  , -0.00427007, -0.02073108,  0.00183165,
        0.00412808,  0.00030917,  0.08338374, -0.00225027,  0.01116387,
       -0.00093402, -0.00107135,  0.00372709,  0.00236066,  0.00010012,
       -0.00470158])
              jac: []
  lagrangian_grad: array([ 0.00508902,  0.000236  , -0.00427007, -0.02073108,  0.00183165,
        0.00412808,  0.00030917,  0.08338374, -0.00225027,  0.01116387,
       -0.00093402, -0.00107135,  0.00372709,  0.00236066,  0.00010012,
       -0.00470158])
          message: '

In [639]:
np.round(array([ 2.07211884e+00, -2.17740576e+00, -2.83192146e+00, -1.40338963e+01,
       -1.15329889e+02, -2.42401483e+01, -1.14517625e+00,  1.00428382e-02,
        3.53300851e+00,  5.23275992e-01,  2.68007969e-01, -8.88934149e-01,
        1.86632165e+00, -3.44450563e-02,  5.33898531e-01,  1.86239381e+03]),4)

array([ 2.0721000e+00, -2.1774000e+00, -2.8319000e+00, -1.4033900e+01,
       -1.1532990e+02, -2.4240100e+01, -1.1452000e+00,  1.0000000e-02,
        3.5330000e+00,  5.2330000e-01,  2.6800000e-01, -8.8890000e-01,
        1.8663000e+00, -3.4400000e-02,  5.3390000e-01,  1.8623938e+03])

In [645]:
1.86239381e+03

1862.39381

In [646]:
np.linalg.norm(array([ 0.00508902,  0.000236  , -0.00427007, -0.02073108,  0.00183165,
        0.00412808,  0.00030917,  0.08338374, -0.00225027,  0.01116387,
       -0.00093402, -0.00107135,  0.00372709,  0.00236066,  0.00010012,
       -0.00470158]))

0.08729604430708358